In [3]:
#importing the required libraries

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import pandas as pd
import matplotlib.pyplot as plt
import platform

In [4]:
#importing the dataset from kaggle
DATASET_NAME = 'rock_paper_scissors'

(train_data, test_data), dataset_info = tfds.load(
    name=DATASET_NAME,
    data_dir='tmp',
    with_info=True,
    as_supervised=True,
    split=[tfds.Split.TRAIN, tfds.Split.TEST],
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/2520 [00:00<?, ? examples/s]

Shuffling tmp/rock_paper_scissors/incomplete.8Z9AUO_3.0.0/rock_paper_scissors-train.tfrecord*...:   0%|       …

Generating test examples...:   0%|          | 0/372 [00:00<?, ? examples/s]

Shuffling tmp/rock_paper_scissors/incomplete.8Z9AUO_3.0.0/rock_paper_scissors-test.tfrecord*...:   0%|        …

Dataset rock_paper_scissors downloaded and prepared to tmp/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.


In [5]:
dataset_info

tfds.core.DatasetInfo(
    name='rock_paper_scissors',
    full_name='rock_paper_scissors/3.0.0',
    description="""
    Images of hands playing rock, paper, scissor game.
    """,
    homepage='http://laurencemoroney.com/rock-paper-scissors-dataset',
    data_dir=PosixGPath('/tmp/tmpssr31do6tfds'),
    file_format=tfrecord,
    download_size=219.53 MiB,
    dataset_size=219.23 MiB,
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=3),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=372, num_shards=1>,
        'train': <SplitInfo num_examples=2520, num_shards=2>,
    },
    citation="""@ONLINE {rps,
    author = "Laurence Moroney",
    title = "Rock, Paper, Scissors Dataset",
    month = "feb",
    year = "2019",
    url = "http://laurencemoroney.com/rock-paper-scissors-dataset"
    }""",
)

In [6]:
NUM_TRAIN_EXAMPLES = dataset_info.splits['train'].num_examples
NUM_TEST_EXAMPLES = dataset_info.splits['test'].num_examples
NUM_CLASSES = dataset_info.features['label'].num_classes

print('Number of TRAIN examples:', NUM_TRAIN_EXAMPLES)
print('Number of TEST examples:', NUM_TEST_EXAMPLES)
print('Number of label classes:', NUM_CLASSES)

Number of TRAIN examples: 2520
Number of TEST examples: 372
Number of label classes: 3


In [7]:
INPUT_IMG_SIZE = dataset_info.features['image'].shape[0]
INPUT_IMG_SHAPE = dataset_info.features['image'].shape

In [8]:
def format_example(image, label):
    image = tf.cast(image, tf.float32)

    image = (image / 255) -0.5
    # Make sure that image has a right size
    #image = tf.image.resize(image, [300, 300])
    return image, label

In [9]:
dataset_train = train_data.map(format_example)
dataset_test = test_data.map(format_example)

print(dataset_train)

<_MapDataset element_spec=(TensorSpec(shape=(300, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [10]:
BATCH_SIZE = 8

dataset_train_shuffled = dataset_train.batch(
    batch_size=BATCH_SIZE
)

dataset_test_shuffled = dataset_test.batch(BATCH_SIZE)

In [11]:
model = tf.keras.models.Sequential()

# First convolution.
#model.add(tf.keras.layers.ZeroPadding2D(padding=(2, 2)))
model.add(tf.keras.layers.Convolution2D(
    input_shape=INPUT_IMG_SHAPE,
    filters=64,
    kernel_size=3,
    activation=tf.keras.activations.relu
))
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(3, 3),
    strides=(2, 2)
))

# Second convolution.
#model.add(tf.keras.layers.ZeroPadding2D(padding=(2, 2)))
model.add(tf.keras.layers.Convolution2D(
    filters=64,
    kernel_size=3,
    activation=tf.keras.activations.relu
))
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(3, 3),
    strides=(2, 2)
))

# Third convolution.
#model.add(tf.keras.layers.ZeroPadding2D(padding=(2, 2)))
model.add(tf.keras.layers.Convolution2D(
    filters=32,
    kernel_size=3,
    activation=tf.keras.activations.relu
))
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(3, 3),
    strides=(2, 2)
))

# Fourth Convulution
model.add(tf.keras.layers.Convolution2D(
    filters=32,
    kernel_size=3,
    activation=tf.keras.activations.relu
))
model.add(tf.keras.layers.MaxPooling2D(
    pool_size=(3, 3),
    strides=(2, 2)
))

# Flatten the results to feed into dense layers.
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))


# neuron layer.
model.add(tf.keras.layers.Dense(
    units=128,
    activation=tf.keras.activations.relu
))

model.add(tf.keras.layers.Dense(
    units=64,
    activation=tf.keras.activations.relu
))

# Output layer.
model.add(tf.keras.layers.Dense(
    units=NUM_CLASSES,
    activation=tf.keras.activations.softmax
))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 148, 148, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 146, 146, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 72, 72, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 70, 70, 32)        18464     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 34, 34, 32)        0

In [13]:
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
#rmsprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0005)

model.compile(
    optimizer= adam_optimizer,
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)

In [14]:
steps_per_epoch = 2520 // 8
validation_steps = 372 // 8

training_history = model.fit(
    x=dataset_train_shuffled.repeat(),
    validation_data=dataset_test_shuffled.repeat(),
    epochs=15,
    steps_per_epoch= steps_per_epoch,
    validation_steps= validation_steps,
    verbose=1
)

Epoch 1/15
315/315 [==============================] - 19s 35ms/step - loss: 0.2449 - accuracy: 0.8952 - val_loss: 0.4029 - val_accuracy: 0.8804
Epoch 2/15
315/315 [==============================] - 10s 31ms/step - loss: 0.0059 - accuracy: 0.9988 - val_loss: 0.4411 - val_accuracy: 0.9103
Epoch 3/15
315/315 [==============================] - 9s 30ms/step - loss: 0.0046 - accuracy: 0.9984 - val_loss: 0.1772 - val_accuracy: 0.9538
Epoch 4/15
315/315 [==============================] - 10s 31ms/step - loss: 4.1662e-04 - accuracy: 1.0000 - val_loss: 0.6235 - val_accuracy: 0.9103
Epoch 5/15
315/315 [==============================] - 11s 35ms/step - loss: 0.0538 - accuracy: 0.9877 - val_loss: 0.2668 - val_accuracy: 0.8995
Epoch 6/15
315/315 [==============================] - 10s 31ms/step - loss: 0.0080 - accuracy: 0.9972 - val_loss: 1.0738 - val_accuracy: 0.7011
Epoch 7/15
315/315 [==============================] - 10s 31ms/step - loss: 0.0016 - accuracy: 0.9992 - val_loss: 0.5103 - val_accura

In [15]:
test_loss, test_accuracy = model.evaluate(
    x=dataset_test.batch(BATCH_SIZE).take(NUM_TEST_EXAMPLES)
)
print(test_loss)
print(test_accuracy)

47/47 [==============================] - 1s 17ms/step - loss: 0.3065 - accuracy: 0.9220
0.30650338530540466
0.9220430254936218
